In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
df = pd.read_csv('/content/drive/MyDrive/diabetes data set/7 feature datasetfinal.csv')

In [60]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16969 entries, 0 to 16968
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   heartRate     16969 non-null  int64
 1   glucose       16969 non-null  int64
 2   Age           16969 non-null  int64
 3   Sweating      16969 non-null  int64
 4   Diabetic      16969 non-null  int64
 5   hypoglycemia  16969 non-null  int64
dtypes: int64(6)
memory usage: 795.5 KB


In [61]:
X = df.drop('hypoglycemia',axis='columns')
y = df['hypoglycemia']

In [62]:
df['hypoglycemia'].value_counts()

0    11822
1     5147
Name: hypoglycemia, dtype: int64

In [63]:
cols_to_scale = ['heartRate','glucose','Age','Sweating']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

In [64]:
df.head()


,heartRate,glucose,Age,Sweating,Diabetic,hypoglycemia
0,0.384615,0.145,0.000000,0.0,0,0
1,0.461538,0.150,0.000000,1.0,0,0
2,0.057692,0.100,0.000000,1.0,0,0
3,0.346154,0.100,0.000000,1.0,0,0
4,0.269231,0.250,0.838235,0.0,0,0


In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [66]:
X = df.drop('hypoglycemia',axis='columns')
y = testLabels = df.hypoglycemia.astype(np.float32)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [67]:
y_train.value_counts()

0.0    9457
1.0    4118
Name: hypoglycemia, dtype: int64

In [68]:
y.value_counts()

0.0    11822
1.0     5147
Name: hypoglycemia, dtype: int64

In [69]:
count_class_0, count_class_1 = df.hypoglycemia.value_counts()

# Divide by class
df_class_0 = df[df['hypoglycemia'] == 0]
df_class_1 = df[df['hypoglycemia'] == 1]

In [70]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.hypoglycemia.value_counts())

Random under-sampling:
1    5147
0    5147
Name: hypoglycemia, dtype: int64


In [71]:
X = df_test_under.drop('hypoglycemia',axis='columns')
y = df_test_under['hypoglycemia']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [72]:
y_train.value_counts()

1    4118
0    4117
Name: hypoglycemia, dtype: int64

In [86]:
xgbc = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, gamma=0, subsample=0.1,colsample_bytree=1, max_depth=3)

In [87]:
xgbc.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.1, verbosity=1)

In [88]:
prediction_xgbc=xgbc.predict(X_test)

In [89]:
print(confusion_matrix(y_test,prediction_xgbc))
print('\n')
print(classification_report(y_test,prediction_xgbc))
print('\n')
print('Accuracy Score: ',round(accuracy_score(y_test,prediction_xgbc), ndigits=2))

[[2365    0]
 [   0 2364]]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2365
           1       1.00      1.00      1.00      2364

    accuracy                           1.00      4729
   macro avg       1.00      1.00      1.00      4729
weighted avg       1.00      1.00      1.00      4729



Accuracy Score:  1.0


In [79]:
# Oversample 1-class and concat the DataFrames of both classes
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.hypoglycemia.value_counts())

Random over-sampling:
1    11822
0    11822
Name: hypoglycemia, dtype: int64


In [80]:
X = df_test_over.drop('hypoglycemia',axis='columns')
y = df_test_over['hypoglycemia']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [81]:
y_train.value_counts()

1    9458
0    9457
Name: hypoglycemia, dtype: int64

In [90]:
xgbc = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, gamma=0, subsample=0.1,colsample_bytree=1, max_depth=3)

In [91]:
xgbc.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.1, verbosity=1)